# Importing Required packages

In [1]:
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd

# Scraping data
 Assuming that data is housed in a table in class wikitable

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
response = urllib.request.urlopen(url)
soup = BeautifulSoup(response, "html")
table=soup.find("table",class_="wikitable")

# Extracting
 Assuming that
    * Not assigned always is noted as "Not assigned'
    * Data is in a table three columns in the order 'Postcode', 'Borough','Neighbourhood'

In [8]:
postcodeList=[]
boroughList=[]
neighbourhoodList=[]

for row in table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3 and cells[1].find(text=True).replace('\n','') != 'Not assigned' :
        postcodeList.append(cells[0].find(text=True).replace('\n',''))
        boroughList.append(cells[1].find(text=True).replace('\n',''))
        if cells[2].find(text=True).replace('\n','')== 'Not assigned' :
            neighbourhoodList.append(cells[1].find(text=True).replace('\n',''))
        else:
            neighbourhoodList.append(cells[2].find(text=True).replace('\n',''))

In [9]:
df=pd.DataFrame(list(zip(postcodeList,boroughList,neighbourhoodList)),columns =['Postal Code', 'Borough','Neighbourhood'])

# Combining Lat long into DF

In [11]:
geodf=pd.read_csv("Geospatial_Coordinates.csv")
finaldf=df.merge(geodf,on="Postal Code")


In [ ]:
V

In [22]:
import folium

long=-79.3832
lat=43.6532

toronto = folium.Map(location=[lat,long], zoom_start=10)

# add markers to map
for lat, lng, label in zip(finaldf['Latitude'], finaldf['Longitude'], finaldf['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='grey',
        fill=True,
        fill_color='#FFFFFF',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto)  
    
toronto